# Langgraph Prebuilt

This noteboook contains the code to run langgraph agent with prebuilt architectures, we look into the React Agents


In [1]:
# Load the environment variables
%load_ext dotenv
%dotenv

In [ ]:
# Tools used by the agent
from langchain_core.tools import tool


@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [ ]:
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel


class WeatherResponse(BaseModel):
    """The response from the weather agent."""

    conditions: str


# Create the agent using the reasoning action agent design
agent = create_react_agent(
    model="openai:gpt-4o",
    tools=[get_weather],
    response_format=WeatherResponse,
    prompt="You are a helpful assistant",
)

# Set recursion limits to avoid infinite loop
max_iterations = 3
recursion_limit = 2 * max_iterations + 1
agent_with_recursion_limit = agent.with_config(recursion_limit=recursion_limit)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: .... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
import logging

logger = logging.getLogger("langgraph")
logger.setLevel(logging.INFO)

# Run the agent
try:
    response = agent_with_recursion_limit.invoke(
        {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
    )
except GraphRecursionError:
    logger.error("Agent stopped due to max iterations.")

# This is how langgraph returns the structured output
response["structured_response"]